<a href="https://colab.research.google.com/github/Dlinz1/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/DlinzMake_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 2*

---

# Make Features 

- Student should be able to understand the purpose of feature engineering
- Student should be able to work with strings in pandas
- Student should be able to work with dates and times in pandas
- Student should be able to modify or create columns of a dataframe using the `.apply()` function


Helpful Links:
- [Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428)
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series
- [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)

# [Objective](#feature-engineering) - The Purpose of Feature Engineering



## Overview

Feature Engineering is the process of using a combination of domain knowledge, creativity and the pre-existing columns of a dataset to create completely new columns.

 Machine Learning models try to detect patterns in the data and then associate those patterns with certain predictions. The hope is that by creating new columns on our dataset that we can expose our model to new patterns in the data so that it can make better and better predictions.

This is largely a matter of understanding how to work with individual columns of a dataframe with Pandas --which is what we'll be practicing today!

## Follow Along

Columns of a dataframe hold each hold a specific type of data. Lets inspect some of the common datatypes found in datasets and then we'll make a new feature on a dataset using pre-existing columns.

In [0]:
import pandas as pd

In [0]:
# DATASET IN US DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT
source_url = ('https://query.data.world/s/kv3vpxr63qi47ml5azowmwvttirn47')

In [0]:
azh = pd.read_csv(source_url)

### Specific Columns hold specific kinds of data

Some columns hold integer values like the `BedroomAbvGr` which stands for "Bedrooms Above Grade." This is the number of non-basement bedrooms in the home.

For more information on specific column meanings view the [data dictionary](https://github.com/ryanleeallred/datasets/blob/master/Ames%20Housing%20Data/data_description.txt).

In [5]:
# Look at the first ten rows of the `BedroomAbvGr` column.
# Looks like integers to me!
azh.sample(10)

,blkgrp,households,area_median_income,blkgrp_median_income_owners,blkgrp_median_income_renters,avg_hh_size_owners,avg_hh_size_renters,commuters_per_hh_owners,commuters_per_hh_renters,pct_renters,gross_hh_density,median_rooms_per_owner_unit,median_rooms_per_renter_unit,pct_detatched_single_family_unit,median_commute_distance,block_denstiy,employment_access_index,local_job_density,local_retail_job_density,retail_access_index,hh_type1_income,hh_type1_size,hh_type1_workers,hh_type1_ht,hh_type1_h,hh_type1_t,hh_type1_autos,hh_type1_vmt,hh_type1_transit_trips,hh_type1_ht_own,hh_type1_h_own,hh_type1_t_own,hh_type1_autos_own,hh_type1_vmt_own,hh_type1_transit_trips_own,hh_type1_ht_rent,hh_type1_h_rent,hh_type1_t_rent,hh_type1_autos_rent,hh_type1_vmt_rent,...,hh_type7_workers,hh_type7_ht,hh_type7_h,hh_type7_t,hh_type7_autos,hh_type7_vmt,hh_type7_transit_trips,hh_type7_ht_own,hh_type7_h_own,hh_type7_t_own,hh_type7_autos_own,hh_type7_vmt_own,hh_type7_transit_trips_own,hh_type7_ht_rent,hh_type7_h_rent,hh_type7_t_rent,hh_type7_autos_rent,hh_type7_vmt_rent,hh_type7_transit_trips_rent,hh_type8_income,hh_type8_size,hh_type8_workers,hh_type8_ht,hh_type8_h,hh_type8_t,hh_type8_autos,hh_type8_vmt,hh_type8_transit_trips,hh_type8_ht_own,hh_type8_h_own,hh_type8_t_own,hh_type8_autos_own,hh_type8_vmt_own,hh_type8_transit_trips_own,hh_type8_ht_rent,hh_type8_h_rent,hh_type8_t_rent,hh_type8_autos_rent,hh_type8_vmt_rent,hh_type8_transit_trips_rent
2813,'040138161001',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3857,'040210020031',566.0,54022.0,27442.65527,9923.47537,3.63,3.45,1.00770,0.37312,0.27385,2.45453,5.9,4.6,68.79536,32.71160,0.31224,2022.59866,0.37803,0.01414,394.06505,54022.0,4.0,2.0,49.75993,23.51288,26.24705,2.25855,29505.60,15.19271,54.29697,27.14450,27.15248,2.39,29505.60,17.23837,37.72945,13.88323,23.84622,1.91,29505.60,...,1.0,55.30006,28.94725,26.35281,1.85581,22890.60,18.07288,60.82470,33.31945,27.50525,1.99,22890.60,18.38760,40.65085,17.35388,23.29697,1.50,22890.60,17.23837,81033.0,4.0,2.0,35.89990,17.32052,18.57938,2.33855,30281.04,13.72877,39.57490,20.36207,19.21282,2.47,30281.04,16.08915,26.15524,9.25549,16.89975,1.99,30281.04,7.46996
1626,'040131167171',832.0,54022.0,85612.21652,41536.00000,2.81,2.48,1.49107,1.06250,1.00000,5.75907,4.0,4.0,0.00000,9.72759,0.06230,36735.59719,4.72048,2.15391,5202.06489,54022.0,4.0,2.0,42.95533,24.98982,17.96552,1.62000,18519.84,86.76649,53.17808,32.49787,20.68021,2.02,18519.84,62.05815,42.95533,24.98982,17.96552,1.62,18519.84,...,1.0,47.33759,31.23698,16.10060,1.20000,12436.20,93.66184,59.52944,39.87228,19.65717,1.62,12436.20,63.78199,47.33759,31.23698,16.10060,1.20,12436.20,93.66184,81033.0,4.0,2.0,29.64200,16.65988,12.98212,1.70000,19582.56,83.89342,41.12500,26.24116,14.88384,2.10,19582.56,61.48354,29.64200,16.65988,12.98212,1.70,19582.56,83.89342
3828,'040210017011',334.0,54022.0,37093.21151,44392.70652,2.13,1.48,0.80000,1.41379,0.08683,0.01378,5.6,5.5,39.50617,29.26660,0.00739,1911.58774,0.00611,0.00000,213.06386,54022.0,4.0,2.0,57.40831,28.98586,28.42245,2.42787,32302.80,12.61739,58.00968,29.36581,28.64386,2.46,32302.80,13.21609,51.08362,24.98982,26.09380,2.09,32302.80,...,1.0,64.27259,35.59811,28.67448,2.02614,24795.72,14.79025,64.97819,36.01277,28.96542,2.06,24795.72,14.93992,56.85163,31.23698,25.61464,1.67,24795.72,13.21609,81033.0,4.0,2.0,40.33210,20.04063,20.29147,2.50787,33998.40,11.84321,40.80855,20.36207,20.44647,2.54,33998.40,12.64147,35.32113,16.65988,18.66125,2.17,33998.40,3.44767
3204,'040190029011',670.0,46443.0,40021.93061,25693.83613,2.20,2.49,0.91611,0.88534,0.59254,3.03703,5.2,4.1,63.63636,5.07261,0.09972,33049.60761,2.63636,0.36612,3423.06603,46443.0,4.0,2.0,54.00608,29.26792,24.73816,1.90521,22380.84,59.27397

Some columns hold float values like the `LotFrontage` column.

In [8]:
# What types are in this column?

azh['avg_hh_size_owners'].dtype

dtype('float64')

Hmmm, do the values above look like floats to you?

They all have .0 on them so technically they're being stored as floats, but *should* they be stored as floats?

Lets see what all of the possible values for this column are.

In [9]:
# Looking at the first ten rows of avg_hh_size_owners columns
azh['avg_hh_size_owners'].head(10)

0    4.46
1    3.89
2    3.90
3    4.32
4    4.31
5    3.64
6    4.34
7    4.23
8    4.17
9    4.94
Name: avg_hh_size_owners, dtype: float64

Looks to me like the `LotFrontage` column originally held integer values but was cast to a `float` meaning that each original integer values was converted to its corresponding float representation. 

Any guesses as to why that would have happened?


HINT: What's the most common `LotFrontage` value for this column?

In [11]:
# NaN is the most common value in this column. What is a NaN
import numpy as np
np.nan


nan

`NaN` stands stands for "Not a Number" and is the default missing value indicator with Pandas. This means there were cells in this column that didn't have a LotFrontage value recorded for those homes. 

This is where domain knowledge starts to come in. Think about the context we're working with here: houses. What might a null or blank cell representing "Linear feet of street connected to property" mean in the context of a housing dataset?

Ok, so maybe it makes seanse to have some NaNs in this column. What is the datatype of a NaN value?

Perhaps some of this data is truly missing or unrecorded data, but sometimes `NaNs` are more likely to indicate something that was "NA" or "Not Applicable" to a particular observation. There could be multiple reasons why there was no value recorded for a particular feature.

Remember, that Pandas tries to maintain a single datatype for all values in a column, and therefore...

In [25]:
import numpy as np

# What is the datatype of NaN?
type(np.nan)

float

The datatype of a NaN is float!  This means that if we have a column of integer values, but the column has even a single `NaN` that column will not be treated with the integer datatype but all of the integers will be converted to floats in order to try and preserve the same datatype throughout the entire column.

You can see already how understanding column datatypes is crucial to understanding how Pandas help us manage our data.

In [14]:
# How much missing data?
azh['avg_hh_size_owners'].isnull().sum()

123

In [16]:
azh.shape

(4180, 188)

In [0]:
# Drop missing data # Before I drop data chk why it's been excluded or missing. Can I replace it with other data, find an avg. 
azh['avg_hh_size_owners'].dropna(inplace=True)

In [20]:
azh['avg_hh_size_owners'].isnull().sum()

0

### Making new Features

Lets slim down the dataset and consider just a few specific columns:

- `TotalBsmtSF`
- `1stFlrSF`
- `2ndFlrSF`
- `SalePrice1`


In [46]:
# I can make a smaller dataframe with a few specific column headers
# by passing a list of column headers inside of the square brackets
smaller=azh[['blkgrp_median_income_owners', 'blkgrp_median_income_renters', 'hh_type1_income']].copy()
smaller.sample(3) # blkgrp_median_income_owners	blkgrp_median_income_renters hh_type1_income

,blkgrp_median_income_owners,blkgrp_median_income_renters,hh_type1_income
2387,68577.75136,49685.83850,54022.0
2066,51365.89282,34918.78451,54022.0
3265,57316.23169,18242.95230,46443.0


### Syntax for creating new columns

When making a new column on a dataframe, we have to use the square bracket syntax of accessing a column. We can't use "dot syntax" here.

In [47]:
smaller['Total Income'] = smaller['blkgrp_median_income_owners'] + smaller['blkgrp_median_income_renters']
smaller.head()

,blkgrp_median_income_owners,blkgrp_median_income_renters,hh_type1_income,Total Income
0,18333.00000,28207.66075,31615.0,46540.66075
1,18223.82631,28039.68310,31615.0,46263.50941
2,18864.99196,25280.25477,31615.0,44145.24673
3,26114.24648,34994.70371,31615.0,61108.95019
4,14296.27346,19157.88971,31615.0,33454.16317


In [48]:
# Lets add up all of the square footage to get a single square footage 
# column for the entire dataset

smaller['Avg Income'] = smaller['hh_type1_income'] / smaller['Total Income']
smaller.sample(5)

,blkgrp_median_income_owners,blkgrp_median_income_renters,hh_type1_income,Total Income,Avg Income
2887,42091.73157,28232.42244,39383.0,70324.15401,0.560021
1361,60423.90044,10249.89823,54022.0,70673.79867,0.764385
847,33252.46628,15672.84075,54022.0,48925.30703,1.104173
439,39668.11404,25634.38846,54022.0,65302.50250,0.827258
4139,31236.65152,16231.18037,41156.0,47467.83189,0.867029


In [49]:
azh.columns

Index(['blkgrp', 'households', 'area_median_income',
       'blkgrp_median_income_owners', 'blkgrp_median_income_renters',
       'avg_hh_size_owners', 'avg_hh_size_renters', 'commuters_per_hh_owners',
       'commuters_per_hh_renters', 'pct_renters',
       ...
       'hh_type8_autos_own', 'hh_type8_vmt_own', 'hh_type8_transit_trips_own',
       'hh_type8_ht_rent', 'hh_type8_h_rent', 'hh_type8_t_rent',
       'hh_type8_autos_rent', 'hh_type8_vmt_rent',
       'hh_type8_transit_trips_rent', 'Total Income'],
      dtype='object', length=189)

In [50]:
azh[['households', 'commuters_per_hh_renters', 'pct_renters', 'commuters_per_hh_owners' ]].sample(5)

,households,commuters_per_hh_renters,pct_renters,commuters_per_hh_owners
3209,NaN,NaN,NaN,NaN
1030,779.0,0.96980,0.78947,1.44862
2544,350.0,0.66482,0.04286,0.55232
3630,663.0,0.99270,0.84615,1.01075
144,849.0,1.74143,0.07538,1.37595


In [0]:
source_url = ('https://query.data.world/s/4ncmoqjyr25ek4hnxr3vxjz5q47dqi')

In [30]:
atlcrm = pd.read_csv(source_url)
atlcrm.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,crime,number,date,location,beat,neighborhood,npu,lat,long
0,0,LARCENY-NON VEHICLE,103040029,10/31/2010,610 SPRING ST NW,509,Downtown,M,33.77101,-84.38895
1,1,AUTO THEFT,103040061,10/31/2010,850 OAK ST SW,401,West End,T,33.74057,-84.41680
2,2,LARCENY-FROM VEHICLE,103040169,10/31/2010,1344 METROPOLITAN PKWY SW,301,Capitol View Manor,X,33.71803,-84.40774
3,3,AUTO THEFT,103040174,10/31/2010,1752 PRYOR RD SW,307,Betmar LaVilla,Y,33.70731,-84.39674
4,4,LARCENY-NON VEHICLE,103040301,10/31/2010,JOHN WESLEY DOBBS AVE NE / CORLEY ST NE,604,Old Fourth Ward,M,33.75947,-84.36626


In [22]:
atlcrm['crime'].value_counts()

LARCENY-FROM VEHICLE    77345
LARCENY-NON VEHICLE     64697
BURGLARY-RESIDENCE      42941
AUTO THEFT              38168
AGG ASSAULT             19133
ROBBERY-PEDESTRIAN      14446
BURGLARY-NONRES          8505
ROBBERY-RESIDENCE        1880
ROBBERY-COMMERCIAL       1855
RAPE                      990
HOMICIDE                  728
Name: crime, dtype: int64

In [26]:
# Creat a new categorical varialble  # np.where is usful for creating a new variable. 
atlcrm['MostEgregious'] = np.where(atlcrm['crime']=='LARCENY-FROM VEHICLE', "yes", "no")
atlcrm['MostEgregious'].value_counts()

no     193343
yes     77345
Name: MostEgregious, dtype: int64

In [28]:
#value counts of a cat var
atlcrm['neighborhood'].value_counts()

Downtown                19743
Midtown                 13732
Old Fourth Ward          8078
West End                 6932
Lenox                    4832
                        ...  
Chattahoochee              12
Mt. Paran Parkway          11
Old Fairburn Village       10
State Facility              5
Englewood Manor             2
Name: neighborhood, Length: 242, dtype: int64

In [33]:
# This method is useful for combining stats to shrink the data
atlcrm['neighborhood1'] = atlcrm['neighborhood'].map({'Downtown':'The Spot',
                                              'Midtown':'The House',
                                              'Old Fourth Ward':'Old Hood',
                                              'West End':'Da Hood Bruh',
                                              'Lenox':'Stay Out',
                                              'Chattahoochee':'Other',
                                              'Mt. Paran Parkway':'Other',
                                              'Old Fairburn Village':'Other',
                                              'State Facility':'Other',
                                              'Englewood Manor':'Other'})
atlcrm['neighborhood1'].value_counts()

The Spot        19743
The House       13732
Old Hood         8078
Da Hood Bruh     6932
Stay Out         4832
Other              40
Name: neighborhood1, dtype: int64

Ok, we have made two new columns on our small dataset.

- What does a **high** `PricePerSqFt` say about a home that the square footage and price alone don't capture as directly?

- What does a **low** `PricePerSqFt` say about a home that the square footage and price alone don't directly capture?



## Challenge

I hope you can see how we have used existing columns to create a new column on a dataset that say something new about our unit of observation. This is what making new features (columns) on a dataset is all about and why it's so essential to data science --particularly predictive modeling "Machine Learning." 

We'll spend the rest of the lecture and assignment today trying to get as good as we can at manipulating (cleaning) and creating new columns on datasets.

# [Objective](#work-with-strings) Work with Strings with Pandas

## Overview

So far we have worked with numeric datatypes (ints and floats) but we haven't worked with any columns containing string values. We can't simply use arithmetic to manipulate string values, so we'll need to learn some more techniques in order to work with this datatype.

## Follow Along

We're going to import a new dataset here to work with. This dataset is from LendingClub and holds information about loans issued in Q4 of 2018. This dataset is a bit messy so it will give us plenty of opportunities to clean up existing columns as well as create new ones.

The `!wget` shell command being used here does exactly the same thing that your browser does when you type a URL in the address. It makes a request or "gets" the file at that address. However, in our case the file isn't a webpage, it's a compressed CSV file. 

Try copying and pasting the URL from below into your browser, did it start an automatic download? Any URLs like this that start automatic downloads when navigated to can be used along with the `!wget` command to bring files directly into your notebook's memory.

### Load a new dataset

We need to use the `!unzip` command to extract the csv from the zipped folder.

We can also use bash/shell commands to look at the raw file using the `!head` and `!tail` commands

As we look at the raw file itself, do you see anything that might cause us trouble as we read in the CSV file to a dataframe?

In [0]:
# Read in the CSV

The extra rows at the top and bottom of the file have done two things:

1) The top row has made it so that the entire dataset is being interpreted as column headers

2) The bottom two rows have been read into the 'id' column and are causing every column to have at least two `NaN` values in it.

In [0]:
# We can fix the header problem by using the 'skiprows' parameter

Lets look at the NaN values of each column so that you can see the problem that the extra rows at the bottom of the file are creating for us

In [0]:
# Sum null values by column and sort from least to greatest

In [0]:
# Address the extra NaNs in each column by skipping the footer as well.

For good measure, we'll also drop some columns that are made up completely of NaN values.

Why might LendingClub have included columns in their dataset that are 100% blank?

### Clean up the `int_rate` column

When we're preparing a dataset for a machine learning model we typically want to represent don't want to leave any string values in our dataset --because it's hard to do math on words. 

Specifically, we have a column that is representing a numeric value, but currently doesn't have a numeric datatype. Lets look at the first 10 values of the `int_rate` column

In [0]:
# Look at the first 10 values of the int_rate column

In [0]:
# Look at a specific value from the int_rate column

Problems that we need to address with this column:

- String column that should be numeric
- Percent Sign `%` included with the number
- Leading space at the beginning of the string

However, we're not going to try and write exactly the right code to fix this column in one go. We're going to methodically build up to the code that will help us address these problems.


In [0]:
# Lets start with just fixing a single string.
# If we can fix one, we can usually fix all of them

In [0]:
# "Cast" the string value to a float

### Write a function to make our solution reusable!

In [0]:
# Write a function that can do what we have written above to any 
# string that is passsed to it.

In [0]:
# Test out our function by calling it on our example

In [0]:
# is the data type correct?

### Apply our solution to every cell in a column

In [0]:
# What type of data is held in our new column?

# Look at the datatypes of the last 5 columns

## Challenge

We can create a new column with our cleaned values or overwrite the original, whatever we think best suits our needs. On your assignment you will take the same approach in trying to methodically build up the complexity of your code until you have a few lines that will work for any cell in a column. At that point you'll contain all of that functionality in a reusable function block and then use the `.apply()` function to... well... apply those changes to an entire column.

# [Objective](#pandas-apply) Modify and Create Columns using `.apply()`



## Overview

We're already seen one example of using the `.apply()` function to clean up a column. Lets see if we can do it again, but this time on a slightly more complicated use case.

Remember, the goal here is to write a function that will work correctly on any **individual** cell of a specific column. Then we can reuse that function on those individual cells of a dataframe column via the `.apply()` function.

Lets clean up the emp_title "Employment Title" column!

## Follow Along

First we'll try and diagnose how bad the problem is and what improvements we might be able to make.

In [0]:
# Look at the top 20 employment titles

In [0]:
# How many different unique employment titles are there currently?

In [0]:
# How often is the employment_title null?

What are some possible reasons as to why a person's employment title may have not been provided?

In [0]:
# Create some examples that represent the cases that we want to clean up

In [0]:
# Write a function to clean up these use cases and increase uniformity.

In [0]:
# list comprehensions can combine function calls and for loops over lists
# into one succinct and fairly readable single line of code.

In [0]:
# We have a function that works as expected. Lets apply it to our column.
# This time we'll overwrite the original column

We can use the same code as we did earlier to see how much progress was made.


In [0]:
# Look at the top 20 employment titles

In [0]:
# How many different unique employment titles are there currently?

In [0]:
# How often is the employment_title null (NaN)?

## Challenge

Using the .apply() function isn't always about creating new columns on a dataframe, we can use it to clean up or modify existing columns as well. 

# [Objective](#dates-and-times) Work with Dates and Times with Pandas

## Overview

Pandas has its own datatype datatype that makes it extremely convenient to convert strings that are in standard date formates to datetime objects and then use those datetime objects to either create new features on a dataframe or work with the dataset in a timeseries fashion. 

This section will demonstrate how to take a column of date strings, convert it to a datetime object and then use the datetime formatting `.dt` to access specific parts of the date (year, month, day) to generate useful columns on a dataframe.

## Follow Along

### Work with Dates 

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

Many of the most useful date columns in this dataset have the suffix `_d` to indicate that they correspond to dates.

We'll use a list comprehension to print them out

Lets look at the string format of the `issue_d` column

Because this string format %m-%y is a common datetime format, we can just let Pandas detect this format and translate it to the appropriate datetime object.

Now we can see that the `issue_d` column has been changed to hold `datetime` objects.

Lets look at one of the cells specifically to see what a datetime object looks like:

You can see how the month and year have been indicated by the strings that were contained in the column previously, and that the rest of the values have been inferred.

We can use the `.dt` accessor to now grab specific parts of the datetime object. Lets grab just the year from the all of the cells in the `issue_d` column

Now the month.

It's just that easy! Now, instead of printing them out, lets add these year and month values as new columns on our dataframe. Again, you'll have to scroll all the way over to the right in the table to see the new columns.

Because all of these dates come from Q4 of 2018, the `issue_d` column isn't all that interesting. Lets look at the `earliest_cr_line` column, which is also a string, but that could be converted to datetime format.

We're going to create a new column called `days_from_earliest_credit_to_issue`

It's a long column header, but think about how valuable this piece of information could be. This number will essentially indicate the length of a person's credit history and if that is correlated with repayment or other factors could be a valuable predictor!

What we're about to do is so cool! Pandas' datetime format is so smart that we can simply use the subtraction operator `-` in order to calculate the amount of time between two dates. 

Think about everything that's going on under the hood in order to give us such straightforward syntax! Handling months of different lengths, leap years, etc. Pandas datetime objects are seriously powerful!

What's oldest credit history that was involved in Q4 2018? 

25,171 days is ~ 68.96 years of credit history!

## Challenge

Pandas' datetime format is so easy to work with that there's really no excuse for not using dates to make features on a dataframe! Get ready to practice more of this on your assignment.